In [3]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import keras

In [4]:

#from tensorflow.python.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
#from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
#from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.python.keras.utils.np_utils import to_categorical

from keras.layers import Bidirectional


from keras.preprocessing.text import Tokenizer

from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
#from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
#from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
#from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

from keras.layers import Bidirectional


from keras.preprocessing.text import Tokenizer
import torch.utils.data as data_utils
import keras


In [5]:
!pip install tokenizers

In [6]:
from tokenizers import BertWordPieceTokenizer

In [7]:
!pip install transformers

In [8]:
#from transformers import*

In [9]:
import transformers
from transformers import BertTokenizer

from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertModel, BertConfig
from transformers import BertModel,  get_linear_schedule_with_warmup
from transformers import BertTokenizer, AdamW


In [10]:

from tensorflow.keras import layers
max_len = 384
configuration = BertConfig()

In [11]:
encoder = TFBertModel.from_pretrained("bert-base-multilingual-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [12]:
def create_model():
    ## BERT encoder
    #encoder = TFBertModel.from_pretrained("bert-base-multilingual-uncased")

    ## QA Model
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
    embedding = encoder(
        input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask
    )[0]

    start_logits = layers.Dense(1, name="start_logit", use_bias=False)(embedding)
    start_logits = layers.Flatten()(start_logits)

    end_logits = layers.Dense(1, name="end_logit", use_bias=False)(embedding)
    end_logits = layers.Flatten()(end_logits)

    start_probs = layers.Activation(keras.activations.softmax)(start_logits)
    end_probs = layers.Activation(keras.activations.softmax)(end_logits)

    model = keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask],
        outputs=[start_probs, end_probs],
    )
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    optimizer = tf.keras.optimizers.Adam(lr=5e-5)
    model.compile(optimizer=optimizer, loss=[loss, loss],metrics=['accuracy'])
    return model

In [13]:
model = create_model()

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 384)]                0         []                            
                                                                                                  
 input_3 (InputLayer)        [(None, 384)]                0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 384)]                0         []                            
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1673564   ['input_1[0][0]',             
 )                           ngAndCrossAttentions(last_   16         'input_3[0][0]',         

In [17]:
df = pd.read_csv('output60.csv')

In [19]:
df.sample(10)

,Unnamed: 0,passage,question,ans_txt,strt_ind
3115,3115,গণিতবিদ চার্লস ব্যাবেজ ১৭৯২১৮৭১কে কম্পিউটার সৃ...,বে সাধারণভাবে আমরা যেকোনাে ইলেকট্রিক জিনিসের য...,প্রভাবজাত,572
1521,1521,১৮১৯ সালে ওয়াশিংটন আর্ভিংয়ের বিখ্যাত রিপ ভ্যান...,একদিন স্ত্রীর সাথে ঝগড়া করতে করতে আর এই জগৎ সং...,নিজের পোষা,206
3709,3709,পৃথিবীতে নিকোলা টেসলা প্রথম শুরু করেছিলেন সহজল...,কার হাত ধরেই শুরু হয়েছে রোবট ও অটোমেশনের যুগ য...,নিকোলা টেসলা,9
1421,1421,গাণিতিক মডেলের ইতিহাস শতাব্দিপ্রাচীন ১৯২০র দশক...,সমীকরণগুলি বিভিন্ন ভৌত রাশির বা কিসের সাথে তাদ...,ফিজিক্যাল কোয়ান্টিটির,509
3607,3607,চোখের নানাবিধ সমস্যা যেমন দূরের বা কাছের দৃষ্ট...,আবিস্কৃত হয়েছে বিশেষ বিশেষ যন্ত্র যার মাধ্যমে...,চোখের,0
3514,3514,চিকিৎসাক্ষেত্রে বিজ্ঞানের ব্যাপক সাফল্য এসেছে ...,রক্তের সঙ্গে নানারকম রাসায়নিক কি মিশিয়ে গবেষ...,যৌগ,476
1649,1649,জেনারেল রিলেটিভিটির সমীকরণগুলো থেকে অনেকগুলো স...,লেখকদের কাল্পনিক দুনিয়ার সৌজন্যে উপন্যাস সিনেম...,প্যারাডক্সগুলো,415
389,389,এর দুতিন বছরের মধ্যে তাঁরা চলে আসেন তাঁর মামা...,কি প্রাবন্ধিক শঙ্করলাল ভট্টাচার্য বলেছেন অসাধা...,চলচ্চিত্র,184
2959,2959,উনিশশ নয় সালের তেসরা মে স্বামীর মৃত্যুর পর আত...,লেখালেখির বাইরে সমাজে বদল আনার দিকে এবং নারীশি...,মন,149
1073,1073,বাংলা বর্ষপঞ্জি অনুযায়ী বছরের প্রথম দুই মাস ব...,এই সময় সূর্যের প্রচন্ড কিসে উত্তপ্ত হয়ে ওঠে ...,তাপে,101


In [20]:
#df['Question'].fillna(df['Passage'], inplace=True)

In [21]:
#df.iloc[394:401]

In [22]:
# i=0
# ans_text=[]
# while i<len(df):
#   print(i)
#   ans_text.append(df[df['Correct'][i]][i])
#   print(df[df['Correct'][i]][i])
#   i+=1
# df['AnsText']=ans_text

In [23]:
character_set_not =  ['\n',
 '!',
 '"',
 '#',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '=',
 '?',
 '[',
 '\\',
 ']',
 '{',
 '}',
 '\xa0',
 '¬',
 '´',
 '·',
 '।',
 '\u09e4',
 '৷',
 '\u200d',
 '–',
 '—',
 '‘',
 '’',
 '“',
 '”',
 '•',
 'ঃ']

In [24]:
def tokenize(s):
    for i in character_set_not:
        s =  s.replace(i, '');
    return  s

In [25]:
#df1=df[['Passage', 'Question','AnsText']]

In [26]:
#df1.columns = ['passage', 'question','ans_txt']

In [27]:
# j=0
# for i in df1['question']:
#   #print(j,type(i))
#   j+=1
# print(df1['question'][495])

In [28]:
df1=df[['passage','question','ans_txt','strt_ind']]

In [29]:
#df1['passage'] = df1['passage'].map(tokenize)

In [30]:
#df1['question'][0] = df1['question'][0].map(tokenize)


In [31]:
#s='বাংলাদেশের আয়তন কত বর্গকিলোমিটার?'

In [32]:
#df1[:3]

In [33]:
#df1['question'] = df1['question'].map(tokenize)

In [34]:
# for i in range(0,3799):
#   df1['question'][i]=tokenize(df1['question'][i])
#   s=df1['question'][i]
#   print(i,"  ",s)



In [35]:
#df1['ans_txt'] = df1['ans_txt'].map(tokenize)

In [36]:
# i=0
# strt_ind=[]
# while i<len(df1):
#   print(i,df1['ans_txt'][i])
#   strt_ind.append(df1['passage'][i].index(df1['ans_txt'][i]))
#   print(df1['ans_txt'][i])
#   i+=1
# df1['strt_ind']=strt_ind

In [37]:
data=df1

In [38]:
data

,passage,question,ans_txt,strt_ind
0,বাংলাদেশ একটি ক্ষুদ্র আয়তনের জনবহুল দেশ এ দেশে...,বাংলাদেশের আয়তন কত বর্গকিলোমিটার,১৪৭৫৭০,52
1,বাংলাদেশ একটি ক্ষুদ্র আয়তনের জনবহুল দেশ এ দেশে...,মোট আয়তনের কত শতাংশ বনভূমি,১৭,93
2,বাংলাদেশ একটি ক্ষুদ্র আয়তনের জনবহুল দেশ এ দেশে...,বাংলাদেশের রাজধানীর নাম কি,ঢাকা,133
3,বাংলাদেশ একটি ক্ষুদ্র আয়তনের জনবহুল দেশ এ দেশে...,বাংলাদেশে মোট কয়টি বিভাগীয় শহর রয়েছে,আটটি,152
4,বাংলাদেশের মোট জনসংখ্যা প্রায় ১৬ কোটির মতো পৃথ...,বাংলাদেশের মোট জনসংখ্যা প্রায় কোটি,১৬,30
...,...,...,...,...
3794,ইউক্রেনকে ঘিরে ইউরোপে উত্তেজনা এখন তুঙ্গে ইউক্...,গত কয়েক দশকের মধ্যে এই প্রথম ইউরোপ আবার কয়টি ...,একটি,382
3795,ইউক্রেনকে ঘিরে ইউরোপে উত্তেজনা এখন তুঙ্গে ইউক্...,গত কয়েক দশকের মধ্যে এই প্রথম ইউরোপ আবার একটি ...,বড়,387
3796,ইউক্রেনকে ঘিরে ইউরোপে উত্তেজনা এখন তুঙ্গে ইউক্...,গত কয়েক দশকের মধ্যে এই প্রথম ইউরোপ আবার একটি ...,বড় আকারের,387
3797,ইউক্রেনকে ঘিরে ইউরোপে উত্তেজনা এখন তুঙ্গে ইউক্...,গত কয়েক দশকের মধ্যে এই প্রথম ইউরোপ আবার একটি ...,যুদ্ধের,398


In [39]:
df1

,passage,question,ans_txt,strt_ind
0,বাংলাদেশ একটি ক্ষুদ্র আয়তনের জনবহুল দেশ এ দেশে...,বাংলাদেশের আয়তন কত বর্গকিলোমিটার,১৪৭৫৭০,52
1,বাংলাদেশ একটি ক্ষুদ্র আয়তনের জনবহুল দেশ এ দেশে...,মোট আয়তনের কত শতাংশ বনভূমি,১৭,93
2,বাংলাদেশ একটি ক্ষুদ্র আয়তনের জনবহুল দেশ এ দেশে...,বাংলাদেশের রাজধানীর নাম কি,ঢাকা,133
3,বাংলাদেশ একটি ক্ষুদ্র আয়তনের জনবহুল দেশ এ দেশে...,বাংলাদেশে মোট কয়টি বিভাগীয় শহর রয়েছে,আটটি,152
4,বাংলাদেশের মোট জনসংখ্যা প্রায় ১৬ কোটির মতো পৃথ...,বাংলাদেশের মোট জনসংখ্যা প্রায় কোটি,১৬,30
...,...,...,...,...
3794,ইউক্রেনকে ঘিরে ইউরোপে উত্তেজনা এখন তুঙ্গে ইউক্...,গত কয়েক দশকের মধ্যে এই প্রথম ইউরোপ আবার কয়টি ...,একটি,382
3795,ইউক্রেনকে ঘিরে ইউরোপে উত্তেজনা এখন তুঙ্গে ইউক্...,গত কয়েক দশকের মধ্যে এই প্রথম ইউরোপ আবার একটি ...,বড়,387
3796,ইউক্রেনকে ঘিরে ইউরোপে উত্তেজনা এখন তুঙ্গে ইউক্...,গত কয়েক দশকের মধ্যে এই প্রথম ইউরোপ আবার একটি ...,বড় আকারের,387
3797,ইউক্রেনকে ঘিরে ইউরোপে উত্তেজনা এখন তুঙ্গে ইউক্...,গত কয়েক দশকের মধ্যে এই প্রথম ইউরোপ আবার একটি ...,যুদ্ধের,398


In [40]:
slow_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
save_path = 'bert-base-multilingual-uncased'
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer('bert-base-multilingual-uncased/vocab.txt', lowercase=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

In [41]:
j=0
max=0
while j<len(df1):
  tokenized = tokenizer.encode(df1['ans_txt'][j])
  ln=len(tokenized)
  if max<ln:
    max=ln
    s=df1['ans_txt'][j]
  j+=1
print(max)
print(s)
type(tokenized)

21
সার জগদীশ চন্দ্র বসু ইন্সটিটিউশন ও কলেজ


tokenizers.Encoding

In [42]:
print('max passage 359')
print('max question 108')
print('max passage+question 409')

max passage 359
max question 108
max passage+question 409


In [43]:
class SquadExample:
    def __init__(self, question, context, start_char_idx, answer_text):
        self.question = question
        self.context = context
        self.start_char_idx = start_char_idx
        self.answer_text = answer_text
        #self.all_answers = all_answers
        self.skip = False
    def preprocess(self):
        context = self.context
        question = self.question
        answer_text = self.answer_text
        start_char_idx = self.start_char_idx

        # Clean context, answer and question
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())
        answer = " ".join(str(answer_text).split())

        end_char_idx = start_char_idx + len(answer)
        if end_char_idx >= len(context):
            self.skip = True
            return
        # Mark the character indexes in context that are in answer
        is_char_in_ans = [0] * len(context)
        for idx in range(start_char_idx, end_char_idx):
            is_char_in_ans[idx] = 1
        tokenized_context = tokenizer.encode(context)

        # Find tokens that were created from answer characters
        ans_token_idx = []
        for idx, (start, end) in enumerate(tokenized_context.offsets):
            if sum(is_char_in_ans[start:end]) > 0:
                ans_token_idx.append(idx)

        if len(ans_token_idx) == 0:
            self.skip = True
            return

        # Find start and end token index for tokens from answer
        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]

        # Tokenize question
        tokenized_question = tokenizer.encode(question)

        # Create inputs
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
        attention_mask = [1] * len(input_ids)

        # Pad and create attention masks.
        # Skip if truncation is needed
        padding_length = max_len - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return

        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.start_token_idx = start_token_idx
        self.end_token_idx = end_token_idx
        self.context_token_to_char = tokenized_context.offsets


In [44]:
j=0
while j<len(df1):
  context = df1['passage'][j]
  question = df1['question'][j]
  answer_text = df1['ans_txt'][j]
  start_char_idx = df1['strt_ind'][j]
  squad_eg = SquadExample(
                    question, context, start_char_idx, answer_text
                )
  squad_eg.preprocess()
  #squad_examples.append(squad_eg)
  j+=1



In [45]:
train, test = train_test_split(df1, test_size=0.25)
val = test

In [46]:
 train['ind']=list(range(0,len(train)))
 train=train.set_index('ind')
 train.head()

,passage,question,ans_txt,strt_ind
ind,,,,
0,চিকিৎসাক্ষেত্রে বিজ্ঞানের ব্যাপক সাফল্য এসেছে ...,চিকিৎসাক্ষেত্রে বিজ্ঞানের ব্যাপক সাফল্য এসেছে ...,শতকে,57
1,জীবনানন্দের প্রথম কাব্যে নজরুল ইসলামের প্রভাব ...,১৯৯৯ খ্রিষ্টাব্দে তাঁর কি পালিত হচ্ছিল,জন্মশতবার্ষিকী,229
2,আক্রান্ত থেকে সুস্থ লোকের শরীরে জীবাণুর সংক্রম...,আক্রান্ত থেকে সুস্থ লোকের শরীরে কিসের সংক্রমণ ...,জীবাণুর,32
3,রবীন্দ্রনাথ ঠাকুর ছিলেন প্রথম অইউরোপীয় যিনি স...,তাঁর কাব্যগ্রন্থ গীতাঞ্জলির ইংরেজি অনুবাদের জন...,অ্যাকাডেমি,146
4,তবুও তাঁর কবি পরিচিতিই তাঁকে বিশ্ববরেণ্য করে ত...,তবুও তাঁর কবি পরিচিতিই তাঁকে কেমন করে তুলেছিল ...,বিশ্ববরেণ্য,29


In [47]:
 test['ind']=list(range(0,len(test)))
 test=test.set_index('ind')
 test.head()

,passage,question,ans_txt,strt_ind
ind,,,,
0,অষ্টাদশ ও ঊনবিংশ শতাব্দীতে ইউরােপসহ সমগ্র বিশ্...,এই বিস্ময় আরও তীব্র হয়েছে বিংশ শতাব্দীতে কাদ...,মানুষের,72
1,১৯৯২ সালে অস্ট্রেলিয়া ও নিউজিল্যান্ড অনুষ্ঠিত...,১৯৯২ সালে কত তম বিশ্বকাপ অনুষ্ঠিত হয়,পঞ্চম,47
2,কলকাতায় অধ্যাপক শিবনারায়ণ রায় বিবিসি বাংলাক...,নজরুল উঠে এসেছিলেন কিসের অতি পেছিয়ে থাকা শ্রে...,সমাজের,143
3,ইউক্রেনকে ঘিরে ইউরোপে উত্তেজনা এখন তুঙ্গে ইউক্...,রাশিয়ার পাল্টা ন্যাটো জোটও তাদের কি বাড়িয়েছে,সামরিক তৎপরতা,197
4,ক্রিকেটের ইতিহাস দীর্ঘদিনের হলেও ১৯৭৫ সাল থেক...,কবে থেকে বিশ্বকাপ ক্রিকেট শুরু হয়,১৯৭৫,34


In [48]:
def create_squad_examples(raw_data):
    squad_examples = []
    j=0

    while j<len(raw_data):
        context = raw_data['passage'][j]
        question = raw_data['question'][j]
        answer_text = raw_data['ans_txt'][j]
        start_char_idx = raw_data['strt_ind'][j]
        squad_eg = SquadExample(
                    question, context, start_char_idx, answer_text
                )
        squad_eg.preprocess()
        squad_examples.append(squad_eg)
        j+=1
    return squad_examples

In [49]:
def create_inputs_targets(squad_examples):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in squad_examples:
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key))
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]]
    return x, y

In [50]:
ddd={
    'input' : [],
     'b': [],
     'c':[],
}

In [51]:
for k in ddd:
  print(k)

input
b
c


In [52]:
train_squad_examples = create_squad_examples(train)
x_train, y_train = create_inputs_targets(train_squad_examples)
print(f"{len(train_squad_examples)} training points created.")

eval_squad_examples = create_squad_examples(test)
x_eval, y_eval = create_inputs_targets(eval_squad_examples)
print(f"{len(eval_squad_examples)} evaluation points created.")

2849 training points created.
950 evaluation points created.


In [53]:
x_train

[array([[  101,   623, 17469, ...,     0,     0,     0],
        [  101,   625, 92814, ...,     0,     0,     0],
        [  101,   608, 91534, ...,     0,     0,     0],
        ...,
        [  101,   633, 95041, ...,     0,     0,     0],
        [  101,   637, 23006, ...,     0,     0,     0],
        [  101,   645, 19353, ...,     0,     0,     0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])]

In [54]:
y_eval

[array([ 40,  17,  59,  93,  14,  33,  14,  34,   4,  30,  57,  11,  93,
        178,   2,  17,  48,  11,   2, 179, 140,  18,   4,  64,  79,  65,
        152,  69,  38,  73,  95,   1,   5, 200,  85,  33,  97, 197,  14,
          1,  11,  35,   6, 123,  65,  24,  56,   1,  30,  55, 227,  47,
          8,  34,  36,  19,  98,  42,  36,  62,  14,  16,  34, 151,  22,
         28,  17,  40,  13,  84,   1,  15,   5,  57,  55,  15,  95,  27,
         22,  17,   5,   6,  60, 103,  25,  37,  17,  47,  16, 223,  46,
        159,  38,  15, 186, 101,  15,  33,  42,  41,  49, 127,  93,  24,
        114,  25,  62,  49,  48,  43,  29,  45,  10,  31,  39,  31, 116,
         44,  45,  99, 185,  35, 125,  43,  74,   5,  34, 154,  52, 192,
         95,  46,  83,  11,  44, 129,   6,  98,  81,  72, 103, 154,  55,
         81,  22,  88, 102,  44,  74,  18,  26,   1,  46, 142,  47,  62,
         77,  53,  35,  28,  27,  33, 207,  29,   9, 128,  60,  31,  85,
         64, 125,  28,  39,  78,  84,  42, 132,  90

In [55]:
def normalize_text(text):
    #text = text.lower()

    # Remove punctuations
    exclude = set(string.punctuation)
    text = "".join(ch for ch in text if ch not in exclude)

    # Remove articles
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    text = re.sub(regex, " ", text)

    # Remove extra white space
    text = " ".join(text.split())
    return text


class ExactMatch(keras.callbacks.Callback):
    """
    Each `SquadExample` object contains the character level offsets for each token
    in its input paragraph. We use them to get back the span of text corresponding
    to the tokens between our predicted start and end tokens.
    All the ground-truth answers are also present in each `SquadExample` object.
    We calculate the percentage of data points where the span of text obtained
    from model predictions matches one of the ground-truth answers.
    """

    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def on_epoch_end(self, epoch, logs=None):
        pred_start, pred_end = self.model.predict(self.x_eval)
        count = 0
        eval_examples_no_skip = [_ for _ in eval_squad_examples if _.skip == False]
        for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
            squad_eg = eval_examples_no_skip[idx]
            offsets = squad_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            if start >= len(offsets):
                continue
            pred_char_start = offsets[start][0]
            if end < len(offsets):
                pred_char_end = offsets[end][1]
                pred_ans = squad_eg.context[pred_char_start:pred_char_end]
            else:
                pred_ans = squad_eg.context[pred_char_start:]

In [57]:
model.fit(x_train,y_train,batch_size=12,epochs=5,verbose=1,validation_data=(x_eval, y_eval))

Epoch 1/5


234/234 [==============================] - 343s 1s/step - loss: 11.9826 - activation_loss: 5.9745 - activation_1_loss: 6.0082 - activation_accuracy: 0.0032 - activation_1_accuracy: 0.0018 - val_loss: 11.9013 - val_activation_loss: 5.9506 - val_activation_1_loss: 5.9506 - val_activation_accuracy: 0.0117 - val_activation_1_accuracy: 0.0064
Epoch 2/5
234/234 [==============================] - 295s 1s/step - loss: 11.9331 - activation_loss: 5.9650 - activation_1_loss: 5.9681 - activation_accuracy: 0.0032 - activation_1_accuracy: 0.0043 - val_loss: 11.9013 - val_activation_loss: 5.9506 - val_activation_1_loss: 5.9506 - val_activation_accuracy: 0.0085 - val_activation_1_accuracy: 0.0085
Epoch 3/5
234/234 [==============================] - 280s 1s/step - loss: 11.9346 - activation_loss: 5.9674 - activation_1_loss: 5.9672 - activation_accuracy: 0.0021 - activation_1_accuracy: 0.0018 - val_loss: 11.9013 - val_activation_loss: 5.9506 - val_activation_1_loss: 5.9506 - val_activation_accuracy: 0.0

In [59]:
# Evaluate the model on the test data
evaluation = model.evaluate(x_eval, y_eval, verbose=1)

# Print the evaluation metrics
print("Test Loss:", evaluation[0])
print("Test Accuracy:", evaluation[1])

30/30 [==============================] - 27s 893ms/step - loss: 11.9013 - activation_loss: 5.9506 - activation_1_loss: 5.9506 - activation_accuracy: 0.0107 - activation_1_accuracy: 0.0096
Test Loss: 11.901284217834473
Test Accuracy: 5.950642108917236


In [1]:
from sklearn.metrics import accuracy_score, precision_score, f1_score

In [ ]:


# Assuming you have already trained your model and obtained predictions on the test data
y_pred = model.predict_classes(x_eval)

# Convert one-hot encoded y_test to categorical labels if needed
y_test_labels = np.argmax(y_eval, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_labels, y_pred)

# Calculate precision
precision = precision_score(y_test_labels, y_pred, average='weighted')

# Calculate F1 score
f1 = f1_score(y_test_labels, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1 Score:", f1)

In [60]:
hist=model.history.history

In [61]:
#f=open('history.json','w')


import json
hist=model.history.history
json_object = json.dumps(hist)

with open("hist.json", "w") as outfile:
    outfile.write(json_object)